In [ ]:
import pandas as pd
import os


In [5]:
fut = pd.read_csv("sensex_future.csv").set_index("date_time")
fut.index = pd.to_datetime(fut.index)
opt = pd.read_csv("sensex.csv").set_index("date_time")
opt.index = pd.to_datetime(opt.index)

In [6]:
fut.head()

,scrip,open,high,low,close,volume,openinterest
date_time,,,,,,,
2024-01-01 09:15:00,SENSEX-I,72218.39,72222.72,72104.77,72143.27,0,0
2024-01-01 09:16:00,SENSEX-I,72142.35,72159.23,72083.18,72087.38,0,0
2024-01-01 09:17:00,SENSEX-I,72089.73,72127.49,72089.73,72119.61,0,0
2024-01-01 09:18:00,SENSEX-I,72124.98,72132.88,72077.93,72103.10,0,0
2024-01-01 09:19:00,SENSEX-I,72101.84,72116.55,72064.69,72068.92,0,0


In [7]:
opt.head()

,index,scrip,open,high,low,close,volume,openinterest,expiry_date,dte
date_time,,,,,,,,,,
2024-01-01 09:15:00,SENSEX,74500CE,35.55,35.55,28.35,28.35,180,0,2024-01-05,4
2024-01-01 09:15:00,SENSEX,74300CE,36.80,38.85,36.80,37.25,100,0,2024-01-05,4
2024-01-01 09:15:00,SENSEX,74200CE,46.25,46.25,40.00,40.00,70,0,2024-01-05,4
2024-01-01 09:15:00,SENSEX,74100CE,57.35,57.35,47.25,47.25,120,0,2024-01-05,4
2024-01-01 09:15:00,SENSEX,74000CE,48.45,54.45,48.45,49.15,360,0,2024-01-05,4


In [17]:
entry_time = pd.to_datetime("2024-01-01 09:20:00")
exit_time = pd.to_datetime("2024-01-01 15:00:00")
sl = 20
dk = 20

In [9]:
entry_close = fut.loc[entry_time, 'close']
entry_close

np.float64(72043.13)

In [10]:
fut_close_entry = round(entry_close / 100) * 100
fut_close_entry

72000

In [12]:
# Create CE and PE strike symbols based on fut_close_entry
ce_strike = f"{int(fut_close_entry)}CE"
pe_strike = f"{int(fut_close_entry)}PE"

# Extract data for CE and PE from entry_time to exit_time
ce_data = opt[(opt['scrip'] == ce_strike) ]
pe_data = opt[(opt['scrip'] == pe_strike) ]

ce_data.head()

,index,scrip,open,high,low,close,volume,openinterest,expiry_date,dte
date_time,,,,,,,,,,
2024-01-01 09:15:00,SENSEX,72000CE,725.0,787.30,640.05,647.55,1070,0,2024-01-05,4
2024-01-01 09:16:00,SENSEX,72000CE,647.5,658.30,605.65,605.65,2440,0,2024-01-05,4
2024-01-01 09:17:00,SENSEX,72000CE,610.0,614.20,556.85,590.30,14300,0,2024-01-05,4
2024-01-01 09:18:00,SENSEX,72000CE,586.3,597.45,570.00,582.70,2480,0,2024-01-05,4
2024-01-01 09:19:00,SENSEX,72000CE,578.8,595.50,574.05,579.05,1710,0,2024-01-05,4


In [15]:
ce_data = ce_data.loc[entry_time:exit_time]
pe_data = pe_data.loc[entry_time:exit_time]



In [16]:
ce_price = ce_data.loc[entry_time, 'close']
pe_price = pe_data.loc[entry_time, 'close']
ce_price,pe_price

(np.float64(564.6), np.float64(402.8))

In [18]:
ce_price * (1 + dk / 100)

np.float64(677.52)

In [19]:
ce_price  + (dk / 100) * ce_price

np.float64(677.52)

In [38]:
ce_price

np.float64(564.6)

In [39]:
dk_price = ce_price * (1 - dk / 100)
sl_price = dk_price * (1 + sl / 100)

In [63]:
ce_price

np.float64(564.6)

In [40]:
dk_price, sl_price

(np.float64(451.68000000000006), np.float64(542.0160000000001))

In [58]:
target_hit_time = ce_data[ce_data['low'] <= dk_price].index.min()


In [59]:
target_hit_time

NaT

In [60]:
if pd.notna(target_hit_time) :
    
    sl_data = ce_data[ce_data.index >= target_hit_time]


    sl_hit_time = sl_data[sl_data['high'] >= sl_price].index.min()

    if (pd.isna(sl_hit_time) ):
        exit_time = ce_data.index.max()
        exit_price = ce_data.loc[exit_time, 'close']
        trade_type = 'target'
        pnl = exit_price - dk_price  # For sell side
    else:
        exit_time = sl_hit_time
        exit_price =  sl_price
        trade_type = 'stoploss'
        pnl = dk_price - exit_price  # For sell side

else :

    target_hit_time = None
    sl_hit_time = None
    pnl = 0
    exit_time = ce_data.index.max()

In [61]:
entry_time, exit_time, target_hit_time, sl_hit_time, trade_type, pnl

(Timestamp('2024-01-01 09:20:00'),
 Timestamp('2024-01-01 15:00:00'),
 None,
 None,
 'exit',
 0)

In [ ]:
def trade_strategy_fast(data, entry_time, exit_time, entry_price, dk, sl):
    """
    Vectorized trade strategy for CE/PE options (SELL side).
    """
    # Initial target and stoploss
    dk_price = entry_price * (1 - dk / 100)
    sl_price = dk_price * (1 + sl / 100)

    # Slice data for the trading window
    data_window = data.loc[entry_time:exit_time]

    # Find first time target (dk) is hit
    target_hit_time = data[data['low'] <= dk_price].index.min()


    if pd.notna(target_hit_time) :
    
        sl_data = data_window[data_window.index >= target_hit_time]
        sl_hit_time = sl_data[sl_data['high'] >= sl_price].index.min()

        if (pd.isna(sl_hit_time) ):
            exit_time = sl_data.index.max()
            exit_price = sl_data.loc[exit_time, 'close']
            trade_type = 'target'
            pnl =  dk_price - exit_price  # For sell side
        else:
            exit_time = sl_hit_time
            exit_price =  sl_price
            trade_type = 'stoploss'
            pnl = dk_price - exit_price  # For sell side

    else :

        target_hit_time = None
        sl_hit_time = None
        pnl = 0
        exit_time = data.index.max()

    return {
        "entry_price": entry_price,
        "dk_price": dk_price,
        "sl_price": sl_price,
        "target_hit_time": target_hit_time,
        "sl_hit_time": sl_hit_time,
        "trade_type": trade_type if pd.notna(target_hit_time) else 'no_trade',
        "pnl": pnl
    }

# Example usage
ce_result = trade_strategy_fast(ce_data, entry_time, exit_time, ce_price, dk, sl)
pe_result = trade_strategy_fast(pe_data, entry_time, exit_time, pe_data.loc[entry_time, 'close'], dk, sl)

print("CE Trade Result:", ce_result)
print("PE Trade Result:", pe_result)
print("Total PnL:", ce_result['pnl'] + pe_result['pnl'])


CE Trade Result: {'entry_price': np.float64(564.6), 'dk_price': np.float64(451.68000000000006), 'sl_price': np.float64(542.0160000000001), 'target_hit_time': None, 'sl_hit_time': None, 'trade_type': 'no_trade', 'pnl': 0}
PE Trade Result: {'entry_price': np.float64(402.8), 'dk_price': np.float64(322.24), 'sl_price': np.float64(386.688), 'target_hit_time': None, 'sl_hit_time': None, 'trade_type': 'no_trade', 'pnl': 0}
Total PnL: 0


In [24]:
ce_result["pnl"]

np.float64(323.43000000000006)